<a href="https://colab.research.google.com/github/sheikmohdimran/Experiments_2020/blob/master/NLP/Pytorch_Google_QA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install -q transformers
!pip install -q iterative-stratification
!unzip google-quest-challenge.zip

     |████████████████████████████████| 481kB 3.3MB/s 
     |████████████████████████████████| 3.1MB 16.2MB/s 
     |████████████████████████████████| 1.0MB 33.4MB/s 
     |████████████████████████████████| 870kB 56.0MB/s 
Archive:  google-quest-challenge.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [0]:
import collections
from datetime import datetime, timedelta
import os
import requests
import threading

_VersionConfig = collections.namedtuple('_VersionConfig', 'wheels,server')
VERSION = "xrt==1.15.0"  #@param ["xrt==1.15.0", "torch_xla==nightly"]
CONFIG = {
    'xrt==1.15.0': _VersionConfig('1.15', '1.15.0'),
    'torch_xla==nightly': _VersionConfig('nightly', 'XRT-dev{}'.format(
        (datetime.today() - timedelta(1)).strftime('%Y%m%d'))),
}[VERSION]
DIST_BUCKET = 'gs://tpu-pytorch/wheels'
TORCH_WHEEL = 'torch-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
TORCH_XLA_WHEEL = 'torch_xla-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)
TORCHVISION_WHEEL = 'torchvision-{}-cp36-cp36m-linux_x86_64.whl'.format(CONFIG.wheels)

# Update TPU XRT version
def update_server_xrt():
  print('Updating server-side XRT to {} ...'.format(CONFIG.server))
  url = 'http://{TPU_ADDRESS}:8475/requestversion/{XRT_VERSION}'.format(
      TPU_ADDRESS=os.environ['COLAB_TPU_ADDR'].split(':')[0],
      XRT_VERSION=CONFIG.server,
  )
  print('Done updating server-side XRT: {}'.format(requests.post(url)))

update = threading.Thread(target=update_server_xrt)
update.start()

# Install Colab TPU compat PyTorch/TPU wheels and dependencies
!pip uninstall -y torch torchvision
!gsutil cp "$DIST_BUCKET/$TORCH_WHEEL" .
!gsutil cp "$DIST_BUCKET/$TORCH_XLA_WHEEL" .
!gsutil cp "$DIST_BUCKET/$TORCHVISION_WHEEL" .
!pip install -q "$TORCH_WHEEL"
!pip install -q "$TORCH_XLA_WHEEL"
!pip install -q "$TORCHVISION_WHEEL"
!sudo apt-get install libomp5
update.join()

Updating server-side XRT to 1.15.0 ...
Uninstalling torch-1.4.0:
Done updating server-side XRT: <Response [200]>
  Successfully uninstalled torch-1.4.0
Uninstalling torchvision-0.5.0:
  Successfully uninstalled torchvision-0.5.0
Copying gs://tpu-pytorch/wheels/torch-1.15-cp36-cp36m-linux_x86_64.whl...
- [1 files][ 77.8 MiB/ 77.8 MiB]                                                
Operation completed over 1 objects/77.8 MiB.                                     
Copying gs://tpu-pytorch/wheels/torch_xla-1.15-cp36-cp36m-linux_x86_64.whl...
- [1 files][109.8 MiB/109.8 MiB]                                                
Operation completed over 1 objects/109.8 MiB.                                    
Copying gs://tpu-pytorch/wheels/torchvision-1.15-cp36-cp36m-linux_x86_64.whl...
/ [1 files][  2.1 MiB/  2.1 MiB]                                                
Operation completed over 1 objects/2.1 MiB.                                      
ERROR: fastai 1.0.60 requires torchvision, which i

In [0]:
import transformers, sys, os, gc
import numpy as np, pandas as pd, math
import torch, random, os, multiprocessing, glob
import torch.nn.functional as F
import torch, time
import torch_xla.core.xla_model as xm

from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit, MultilabelStratifiedKFold
from scipy.stats import spearmanr
from torch import nn
from torch.utils import data
from torch.utils.data import DataLoader, Dataset
from transformers import (
    BertTokenizer, BertModel, BertForSequenceClassification, BertConfig,
    WEIGHTS_NAME, CONFIG_NAME, AdamW, get_linear_schedule_with_warmup, 
    get_cosine_schedule_with_warmup,
)

from tqdm import tqdm_notebook as tqdm
print(transformers.__version__)


2.4.1


In [0]:
train = pd.read_csv("./train.csv",)
test = pd.read_csv("./test.csv",)
SEED = 4

In [0]:

target_cols = ['question_asker_intent_understanding', 'question_body_critical', 
               'question_conversational', 'question_expect_short_answer', 
               'question_fact_seeking', 'question_has_commonly_accepted_answer', 
               'question_interestingness_others', 'question_interestingness_self', 
               'question_multi_intent', 'question_not_really_a_question', 
               'question_opinion_seeking', 'question_type_choice',
               'question_type_compare', 'question_type_consequence',
               'question_type_definition', 'question_type_entity', 
               'question_type_instructions', 'question_type_procedure', 
               'question_type_reason_explanation', 'question_type_spelling', 
               'question_well_written', 'answer_helpful',
               'answer_level_of_information', 'answer_plausible', 
               'answer_relevance', 'answer_satisfaction', 
               'answer_type_instructions', 'answer_type_procedure', 
               'answer_type_reason_explanation', 'answer_well_written']

# Ref From SO
# Make sure to end with bcolors.ENDC otherwise color is used in the rest of the terminal
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKGREEN = '\033[92m'
    WARNING = '\033[35m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

# From the Ref Kernel's
from math import floor, ceil

def _get_masks(tokens, max_seq_length):
    """Mask for padding"""
    if len(tokens)>max_seq_length:
        raise IndexError("Token length more than max seq length!")
    return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

def _get_segments(tokens, max_seq_length):
    """Segments: 0 for the first sequence, 1 for the second"""
    
    if len(tokens) > max_seq_length:
        raise IndexError("Token length more than max seq length!")
        
    segments = []
    first_sep = True
    current_segment_id = 0
    
    for token in tokens:
        segments.append(current_segment_id)
        if token == "[SEP]":
            if first_sep:
                first_sep = False 
            else:
                current_segment_id = 1
    return segments + [0] * (max_seq_length - len(tokens))

def _get_ids(tokens, tokenizer, max_seq_length):
    """Token ids from Tokenizer vocab"""
    
    token_ids = tokenizer.convert_tokens_to_ids(tokens)
    input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
    return input_ids

def _trim_input(title, question, answer, max_sequence_length=512, t_max_len=30, q_max_len=239, a_max_len=239):
    
    #293+239+30 = 508 + 4 = 512
    
    t = tokenizer.tokenize(title)
    q = tokenizer.tokenize(question)
    a = tokenizer.tokenize(answer)
    
    t_len = len(t)
    q_len = len(q)
    a_len = len(a)

    if (t_len+q_len+a_len+4) > max_sequence_length:
        
        if t_max_len > t_len:
            t_new_len = t_len
            a_max_len = a_max_len + floor((t_max_len - t_len)/2)
            q_max_len = q_max_len + ceil((t_max_len - t_len)/2)
        else:
            t_new_len = t_max_len
      
        if a_max_len > a_len:
            a_new_len = a_len 
            q_new_len = q_max_len + (a_max_len - a_len)
        elif q_max_len > q_len:
            a_new_len = a_max_len + (q_max_len - q_len)
            q_new_len = q_len
        else:
            a_new_len = a_max_len
            q_new_len = q_max_len
            
            
        if t_new_len+a_new_len+q_new_len+4 != max_sequence_length:
            raise ValueError("New sequence length should be %d, but is %d"%(max_sequence_length, (t_new_len + a_new_len + q_new_len + 4)))
        
        t = t[:t_new_len]
        q = q[:q_new_len]
        a = a[:a_new_len]
    
    return t, q, a

def _convert_to_bert_inputs(title, question, answer, tokenizer, max_sequence_length):
    """Converts tokenized input to ids, masks and segments for BERT"""
    
    #stoken = ["[CLS]"] + title + ["[SPECIAL TOKEN]"] + question + ["[SEP]"] + answer + ["[SEP]"]
    stoken = ["[CLS]"] + title + ["[SPECIAL TOKEN]"] + question + ["[ANS]"] + answer + ["[SEP]"]

    input_ids = _get_ids(stoken, tokenizer, max_sequence_length)
    input_masks = _get_masks(stoken, max_sequence_length)
    input_segments = _get_segments(stoken, max_sequence_length)

    return [input_ids, input_masks, input_segments]


In [0]:
def compute_input_arays(df, columns, tokenizer, max_sequence_length):
 
    input_ids, input_masks, input_segments = [], [], []
    for _, instance in tqdm(df[columns].iterrows()):
        t, q, a = instance.question_title, instance.question_body, instance.answer
        t, q, a = _trim_input(t, q, a, max_sequence_length)
        ids, masks, segments = _convert_to_bert_inputs(t, q, a, tokenizer, max_sequence_length)
        input_ids.append(ids)
        input_masks.append(masks)
        input_segments.append(segments)
    return [
        torch.from_numpy(np.asarray(input_ids, dtype=np.int32)).long(), 
        torch.from_numpy(np.asarray(input_masks, dtype=np.int32)).long(),
        torch.from_numpy(np.asarray(input_segments, dtype=np.int32)).long(),
    ]

def compute_output_arrays(df, columns):
    return np.asarray(df[columns])

class QuestDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, lengths, labels = None):
        
        self.inputs = inputs
        if labels is not None:
            self.labels = labels
        else:
            self.labels = None
        self.lengths = lengths

    def __getitem__(self, idx):
        
        input_ids       = self.inputs[0][idx]
        input_masks     = self.inputs[1][idx]
        input_segments  = self.inputs[2][idx]
        lengths         = self.lengths[idx]
        if self.labels is not None: # targets
            labels = self.labels[idx]
            return input_ids, input_masks, input_segments, labels, lengths
        return input_ids, input_masks, input_segments, lengths

    def __len__(self):
        return len(self.inputs[0])



In [0]:
BATCH_SIZE=12
def train_model(train_loader, optimizer, criterion, scheduler):
    
    model.train()
    avg_loss = 0.
    tk0 = tqdm(enumerate(train_loader))
    
    for idx, batch in tk0:
        
        input_ids, input_masks, input_segments, labels, _ = batch
        input_ids, input_masks, input_segments, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device), labels.to(device)            
        
        output_train = model(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                            )
#        print('output_train', len(output_train))
        logits = output_train[0] #output preds
        
        loss = criterion(logits, labels)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        xm.optimizer_step(optimizer, barrier=True) #optimizer.step()
        scheduler.step() # the position of this depends on the scheduler you use
        optimizer.zero_grad()
        
        avg_loss += loss.item() / len(train_loader)
        del input_ids, input_masks, input_segments, labels

    torch.cuda.empty_cache()
    gc.collect()
    return avg_loss

def val_model(val_loader, val_shape, batch_size=BATCH_SIZE):

    avg_val_loss = 0.
    model.eval() # eval mode
    
    valid_preds = np.zeros((val_shape, 30))
    original = np.zeros((val_shape, 30))
    
    tk0 = tqdm(enumerate(val_loader))
    with torch.no_grad():
        
        for idx, batch in tk0:
            input_ids, input_masks, input_segments, labels, _ = batch
            input_ids, input_masks, input_segments, labels = input_ids.to(device), input_masks.to(device), input_segments.to(device), labels.to(device)            
            
            output_val = model(input_ids = input_ids.long(),
                             labels = None,
                             attention_mask = input_masks,
                             token_type_ids = input_segments,
                            )
            logits = output_val[0] #output preds
            
            avg_val_loss += criterion(logits, labels).item() / len(val_loader)
            valid_preds[idx*batch_size : (idx+1)*batch_size] = logits.detach().cpu().squeeze().numpy()
            original[idx*batch_size : (idx+1)*batch_size]    = labels.detach().cpu().squeeze().numpy()
        
        score = 0
        preds = torch.sigmoid(torch.tensor(valid_preds)).numpy()
        
        rho_val = np.mean([spearmanr(original[:, i], preds[:,i]).correlation for i in range(preds.shape[1])])
        print('\r val_spearman-rho: %s' % (str(round(rho_val, 5))), end = 100*' '+'\n')
        
        for i in range(30):
            #print(target_cols[i], spearmanr(original[:,i], preds[:,i]))
            score += np.nan_to_num(spearmanr(original[:, i], preds[:, i]).correlation)
    return avg_val_loss, score/30.

def predict_result(model, test_loader, batch_size=32):
    
    test_preds = np.zeros((len(test), 30))
    
    model.eval();
    tk0 = tqdm(enumerate(test_loader))
    for idx, x_batch in tk0:
        with torch.no_grad():
            outputs = model(input_ids = x_batch[0].to(device), 
                            labels = None, 
                            attention_mask = x_batch[1].to(device),
                            token_type_ids = x_batch[2].to(device),
                           )
            predictions = outputs[0]
            test_preds[idx*batch_size : (idx+1)*batch_size] = predictions.detach().cpu().squeeze().numpy()

    output = torch.sigmoid(torch.tensor(test_preds)).numpy()        
    return output




In [0]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
input_categories = list(train.columns[[1,2,5]]); input_categories

['question_title', 'question_body', 'answer']

In [0]:
bert_model_config = './bert_config.json'
#bert_config = BertConfig.from_json_file(bert_model_config)
bert_config = BertConfig(pad_token="[ANS]").from_json_file(bert_model_config)
bert_config.num_labels = 30

#bert_model = 'bert-base-uncased'
#do_lower_case = 'uncased' in bert_model
device = xm.xla_device() #device = 'cuda' if torch.cuda.is_available() else 'cpu'
#output_model_file = 'bert_pytorch.bin'

In [0]:
test_inputs = compute_input_arays(test, input_categories, tokenizer, max_sequence_length=512)
lengths_test = np.argmax(test_inputs[0] == 0, axis=1)
lengths_test[lengths_test == 0] = test_inputs[0].shape[1]

#print(do_lower_case, bert_model, device, output_model_file)

NUM_FOLDS = 5  # change this
SEED = 4
#BATCH_SIZE = 12
epochs = 6     # change this
ACCUM_STEPS = 1

kf = MultilabelStratifiedKFold(n_splits = NUM_FOLDS, random_state = SEED)

test_set = QuestDataset(inputs=test_inputs, lengths=lengths_test, labels=None)
test_loader  = DataLoader(test_set, batch_size=32, shuffle=False)
result = np.zeros((len(test), 30))

y_train = train[target_cols].values # dummy

print(bcolors.FAIL, f"For Every Fold, Train {epochs} Epochs", bcolors.ENDC)


 For Every Fold, Train 6 Epochs 


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [0]:
class LogCoshLoss(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, y_t, y_prime_t):
        ey_t = y_t - y_prime_t
        return torch.mean(torch.log(torch.cosh(ey_t + 1e-12)))

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_all(SEED)        

In [0]:
#!pip install -q torchsnooper

In [0]:
#import torchsnooper

#@torchsnooper.snoop()
class BERTBaseUncased(nn.Module):
	def __init__(self):#,batch_size):
		super(BERTBaseUncased, self).__init__()
		self.bert=transformers.BertModel.from_pretrained('bert-base-uncased', output_hidden_states=True)
		self.pre_classifier = nn.Linear(3072, 768)		
		self.bert_drop=nn.Dropout(0.2)
#		self.batch_size = batch_size
		self.classifier = nn.Linear(768,30)
#		self.gap = nn.AdaptiveAvgPool2d((batch_size,None))
		nn.init.xavier_normal_(self.classifier.weight)
		nn.init.xavier_normal_(self.pre_classifier.weight)
		
	def forward(self, input_ids,labels, attention_mask, token_type_ids):
		_,_,out = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
		#hidden_states = o2[2]
		out = torch.cat(tuple([out[i] for i in [-1, -2, -3, -4]]), dim=-1)
		attention_mask = attention_mask.unsqueeze(2)
		out = torch.mul(out, attention_mask)
		out = out[:, 0, :]
		#out = torch.mean(out.view(out.size(0), out.size(2), -1), dim=2)#self.gap(out)
		out=self.pre_classifier(out)
		out = self.bert_drop(out)
		out = self.classifier(out)
		return out


In [0]:
'''#m = nn.AdaptiveMaxPool2d((1, None))
#input = torch.randn(8, 512, 3072)
#output = m(input)
#output.size()
from transformers import BertPreTrainedModel

@torchsnooper.snoop()
class BertForSequenceClassification(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.num_labels = config.num_labels

        self.bert = BertModel(config)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        self.classifier = nn.Linear(config.hidden_size, self.config.num_labels)

        self.init_weights()

#[docs]
#    @add_start_docstrings_to_callable(BERT_INPUTS_DOCSTRING)
    def forward(
        self,
        input_ids=None,
        attention_mask=None,
        token_type_ids=None,
        position_ids=None,
        head_mask=None,
        inputs_embeds=None,
        labels=None,
    ):
        r"""
        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`, defaults to :obj:`None`):
            Labels for computing the sequence classification/regression loss.
            Indices should be in :obj:`[0, ..., config.num_labels - 1]`.
            If :obj:`config.num_labels == 1` a regression loss is computed (Mean-Square loss),
            If :obj:`config.num_labels > 1` a classification loss is computed (Cross-Entropy).

    Returns:
        :obj:`tuple(torch.FloatTensor)` comprising various elements depending on the configuration (:class:`~transformers.BertConfig`) and inputs:
        loss (:obj:`torch.FloatTensor` of shape :obj:`(1,)`, `optional`, returned when :obj:`label` is provided):
            Classification (or regression if config.num_labels==1) loss.
        logits (:obj:`torch.FloatTensor` of shape :obj:`(batch_size, config.num_labels)`):
            Classification (or regression if config.num_labels==1) scores (before SoftMax).
        hidden_states (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_hidden_states=True``):
            Tuple of :obj:`torch.FloatTensor` (one for the output of the embeddings + one for the output of each layer)
            of shape :obj:`(batch_size, sequence_length, hidden_size)`.

            Hidden-states of the model at the output of each layer plus the initial embedding outputs.
        attentions (:obj:`tuple(torch.FloatTensor)`, `optional`, returned when ``config.output_attentions=True``):
            Tuple of :obj:`torch.FloatTensor` (one for each layer) of shape
            :obj:`(batch_size, num_heads, sequence_length, sequence_length)`.

            Attentions weights after the attention softmax, used to compute the weighted average in the self-attention
            heads.

    Examples::

        from transformers import BertTokenizer, BertForSequenceClassification
        import torch

        tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
        model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

        input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
        labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
        outputs = model(input_ids, labels=labels)

        loss, logits = outputs[:2]

        """

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
        )

        pooled_output = outputs[1]

        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)

        outputs = (logits,) + outputs[2:]  # add hidden states and attention if they are here

        if labels is not None:
            if self.num_labels == 1:
                #  We are doing regression
                loss_fct = MSELoss()
                loss = loss_fct(logits.view(-1), labels.view(-1))
            else:
                loss_fct = CrossEntropyLoss()
                loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)

'''

'#m = nn.AdaptiveMaxPool2d((1, None))\n#input = torch.randn(8, 512, 3072)\n#output = m(input)\n#output.size()\nfrom transformers import BertPreTrainedModel\n\n@torchsnooper.snoop()\nclass BertForSequenceClassification(BertPreTrainedModel):\n    def __init__(self, config):\n        super().__init__(config)\n        self.num_labels = config.num_labels\n\n        self.bert = BertModel(config)\n        self.dropout = nn.Dropout(config.hidden_dropout_prob)\n        self.classifier = nn.Linear(config.hidden_size, self.config.num_labels)\n\n        self.init_weights()\n\n#[docs]\n#    @add_start_docstrings_to_callable(BERT_INPUTS_DOCSTRING)\n    def forward(\n        self,\n        input_ids=None,\n        attention_mask=None,\n        token_type_ids=None,\n        position_ids=None,\n        head_mask=None,\n        inputs_embeds=None,\n        labels=None,\n    ):\n        r"""\n        labels (:obj:`torch.LongTensor` of shape :obj:`(batch_size,)`, `optional`, defaults to :obj:`None`):\n   

In [0]:
'''inputs=torch.rand([3,8, 512]).to(device)

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=bert_config).to(device)
output_train = model(input_ids = inputs[0].long(),
                             labels = None,
                             attention_mask = inputs[1],
                             token_type_ids = inputs[2].long(),
                            )
output_train[0].shape'''

"inputs=torch.rand([3,8, 512]).to(device)\n\nmodel = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=bert_config).to(device)\noutput_train = model(input_ids = inputs[0].long(),\n                             labels = None,\n                             attention_mask = inputs[1],\n                             token_type_ids = inputs[2].long(),\n                            )\noutput_train[0].shape"

In [0]:
'''inputs=torch.rand([3,8, 512]).to(device)
model = BERTBaseUncased().to(device)
output_train = model(input_ids = inputs[0].long(),
                             labels = None,
                             attention_mask = inputs[1],
                             token_type_ids = inputs[2].long(),
                            )
output_train[0].shape'''

'inputs=torch.rand([3,8, 512]).to(device)\nmodel = BERTBaseUncased().to(device)\noutput_train = model(input_ids = inputs[0].long(),\n                             labels = None,\n                             attention_mask = inputs[1],\n                             token_type_ids = inputs[2].long(),\n                            )\noutput_train[0].shape'

In [0]:
'''#m = nn.AdaptiveMaxPool3d((7, None, None))
x = torch.randn(1, 512, 3072)
#output = m(input)
#output.size()


import torch.nn.functional as F
#x = F.adaptive_avg_pool2d(x, (1, 1))
x = torch.mean(x.view(x.size(0), x.size(2), -1), dim=2)
x.size()'''

'#m = nn.AdaptiveMaxPool3d((7, None, None))\nx = torch.randn(1, 512, 3072)\n#output = m(input)\n#output.size()\n\n\nimport torch.nn.functional as F\n#x = F.adaptive_avg_pool2d(x, (1, 1))\nx = torch.mean(x.view(x.size(0), x.size(2), -1), dim=2)\nx.size()'

In [0]:
for fold, (train_index, val_index) in enumerate(kf.split(train.values, y_train)):

    print(bcolors.HEADER, "Current Fold:", fold, bcolors.ENDC)

    train_df, val_df = train.iloc[train_index], train.iloc[val_index]
    print("Train and Valid Shapes are", train_df.shape, val_df.shape)
    
    print(bcolors.HEADER, "Preparing train datasets....", bcolors.ENDC)
    
    inputs_train = compute_input_arays(train_df, input_categories, tokenizer, max_sequence_length=512)
    outputs_train = compute_output_arrays(train_df, columns = target_cols)
    outputs_train = torch.tensor(outputs_train, dtype=torch.float32)
    lengths_train = np.argmax(inputs_train[0] == 0, axis=1)
    lengths_train[lengths_train == 0] = inputs_train[0].shape[1]
    
    print(bcolors.HEADER, "Preparing Valid datasets....", bcolors.ENDC)
    
    inputs_valid = compute_input_arays(val_df, input_categories, tokenizer, max_sequence_length=512)
    outputs_valid = compute_output_arrays(val_df, columns = target_cols)
    outputs_valid = torch.tensor(outputs_valid, dtype=torch.float32)
    lengths_valid = np.argmax(inputs_valid[0] == 0, axis=1)
    lengths_valid[lengths_valid == 0] = inputs_valid[0].shape[1]
    
    print(bcolors.HEADER, "Preparing Dataloaders Datasets....", bcolors.ENDC)

    train_set    = QuestDataset(inputs=inputs_train, lengths=lengths_train, labels=outputs_train)
    train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
    #0len(train_loader)
    
    valid_set    = QuestDataset(inputs=inputs_valid, lengths=lengths_valid, labels=outputs_valid)
    valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
    
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=bert_config);
    #for param in model.bert.parameters():
    #  param.requires_grad = False
    #model = BERTBaseUncased().to(device)
    model.zero_grad();
    model.to(device);
    torch.cuda.empty_cache()
    model.train();
    
    i = 0
    best_avg_loss   = 100.0
    best_score      = -1.
    best_param_loss = None
    best_param_score = None

    #optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5, eps=4e-7)
    lrlast = 3e-3
    lrmain = 3e-5
    eps = 4e-8
    optimizer = AdamW(
    [
        {"params":model.bert.parameters(),"lr": lrmain,"eps": eps},
        {"params":model.classifier.parameters(), "lr": lrlast,"eps": eps},       
    ])

    criterion = LogCoshLoss()#nn.BCEWithLogitsLoss()##
    scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=0.05, num_training_steps= epochs*len(train_loader)//ACCUM_STEPS)
    
    print("Training....")
    
    for epoch in tqdm(range(epochs)):
        
        torch.cuda.empty_cache()
        
        start_time   = time.time()
        #if epoch >5:
        #  for param in model.bert.parameters():
         #   param.requires_grad = False
        avg_loss     = train_model(train_loader, optimizer, criterion, scheduler)
        avg_val_loss, score = val_model(valid_loader, val_shape=val_df.shape[0])
        elapsed_time = time.time() - start_time

        print(bcolors.WARNING, 'Epoch {}/{} \t loss={:.4f} \t val_loss={:.4f} \t score={:.6f} \t time={:.2f}s'.format(
            epoch + 1, epochs, avg_loss, avg_val_loss, score, elapsed_time),
        bcolors.ENDC
        )

        if best_avg_loss > avg_val_loss:
            i = 0
            best_avg_loss = avg_val_loss 
            best_param_loss = model.state_dict()

        if best_score < score:
            best_score = score
            best_param_score = model.state_dict()
        else:
            i += 1

    model.load_state_dict(best_param_score)
    result += predict_result(model, test_loader)
    #torch.save(best_param_score, 'best_param_score_{}.pt'.format(fold+1))
    xm.save(best_param_score, 'best_param_score_{}.pt'.format(fold+1))

    del train_df, val_df, model, optimizer, criterion, scheduler
    torch.cuda.empty_cache()
    del valid_loader, train_loader, valid_set, train_set
    torch.cuda.empty_cache()
    torch.cuda.empty_cache()
    gc.collect()

result /= NUM_FOLDS
print(result)


 Current Fold: 0 
Train and Valid Shapes are (4862, 41) (1217, 41)
 Preparing train datasets.... 


 Preparing Valid datasets.... 


 Preparing Dataloaders Datasets.... 
Training....


 val_spearman-rho: 0.30134                                                                                                    
 Epoch 1/6 	 loss=0.0308 	 val_loss=0.0227 	 score=0.301339 	 time=388.74s 


 val_spearman-rho: 0.35644                                                                                                    
 Epoch 2/6 	 loss=0.0217 	 val_loss=0.0210 	 score=0.356436 	 time=350.27s 


...

 val_spearman-rho: 0.36991                                                                                                    
 Epoch 3/6 	 loss=0.0192 	 val_loss=0.0206 	 score=0.369911 	 time=350.99s 


 val_spearman-rho: 0.38045                                                                                                    
 Epoch 4/6 	 loss=0.0171 	 val_loss=0.0197 	 score=0.380452 	 time=351.26s 


 val_spearman-rho: 0.38108                                                                                                    
 Epoch 5/6 	 loss=0.0153 	 val_loss=0.0196 	 score=0.381076 	 time=351.32s 


 val_spearman-rho: 0.38229                                                                                                    
 Epoch 6/6 	 loss=0.0143 	 val_loss=0.0198 	 score=0.382286 	 time=351.02s 


[[0.71974118 0.655289   0.5459225  ... 0.50943775 0.71558447 0.71389556]
 [0.71138033 0.63119755 0.50155121 ... 0.5287067  0.51077296 0.71075975]
 [0.71600377 0.65410678 0.49928364 ... 0.52903548 0.71951439 0.71769613]
 ...
 [0.6987691  0.59215479 0.5085746  ... 0.53092214 0.63309984 0.71553293]
 [0.71160605 0.67950499 0.50843097 ... 0.52766225 0.71068753 0.72275367]
 [0.71361681 0.65499355 0.50210789 ... 0.52611877 0.53830356 0.70977218]]


In [0]:
submission = pd.read_csv(r'./sample_submission.csv')
submission.loc[:, 'question_asker_intent_understanding':] = result
submission.to_csv('submission.csv', index=False)
submission.head()

In [0]:
!cp best_param_score_* ./drive/My\ Drive/

In [0]:
submission1=submission

In [0]:
import numpy as np
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


In [0]:
for i in range(len(target_cols)):
  submission1[target_cols[i]]=submission1[target_cols[i]].apply(lambda x: find_nearest(train[target_cols[i]].unique(), x))

In [0]:
submission1.to_csv('submission1.csv', index=False)